### Natural Processing Language UFPB - [2022.1] </br> 

#### Pratice - Naive Bayes for Sentiment Analyzer  

***


**[*] Description -** The objective of this exercise is to train and evaluate a Naive Bayes classifier to a sentiment analysis task. <br> <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Text used in training and testing must be preprocessed to include the prefix “NAO_” according to the following rule: <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; [#] The prefix “NAO_” must be added to each word after a negation token (no, nor, never, never, either) until next time punctuation. Scores are considered: **. , ? ! ;** <br> <br> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; The training should be performed with 80% of the data and the test with 20%. You must calculate the accuracy of the classifier applied to the test data and compare with the result of Naive Bayes without using the prefix “NAO_”.